## Web scrapping de IMDB

Descarga la información correspondiente y guarda en un dataframe el top de las 250 películas mediante webscrapping. La dirección es https://www.imdb.com/chart/top/?ref_=nv_mv_250

Obtén:
* Título
* Año
* Duración
* Posición
* Rating

A veces las páginas web se protegen de las arañas con lo que deberéis simular ser un usuario... podéis instalar

`!pip install fake-useragent`

Y luego invocar

```py
from fake_useragent import UserAgent
ua = UserAgent()
headers = {'User-Agent': ua.random}
response = requests.get(url, headers=headers)

```

In [ ]:
#!pip install fake-useragent

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/161.7 kB ? eta -:--:--
   --------- ----------------------------- 41.0/161.7 kB 991.0 kB/s eta 0:00:01
   ---------------------- ----------------- 92.2/161.7 kB 1.3 MB/s eta 0:00:01
   ------------------------------------- -- 153.6/161.7 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 161.7/161.7 kB 1.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
# from splinter import Browser
import numpy as np

pd.set_option('max_colwidth', 800)

In [9]:
from fake_useragent import UserAgent
ua = UserAgent()
headers = {'User-Agent': ua.random}


In [10]:
url ="https://www.imdb.com/chart/top/?ref_=nv_mv_250"
response = requests.get(url, headers= headers)


In [11]:
print(response)

<Response [200]>


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re # Importamos el módulo de expresiones regulares

def scrape_imdb_top_250():
    """
    Extrae información de las 250 mejores películas de IMDb y la devuelve en un DataFrame de Pandas.
    """
    url = "https://www.imdb.com/es-es/chart/top/?ref_=nv_mv_250"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept-Language": "es-ES,es;q=0.9,en;q=0.8" # Solicitamos la página en español
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Lanza un error para respuestas HTTP malas (4xx o 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener la página: {e}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    titulos = []
    anos = []
    duraciones = []
    posiciones = []
    ratings = []

    # IMDb envuelve la lista de películas en un contenedor.
    # Basándonos en la estructura de IMDb, buscamos los elementos 'li' que representan cada película.
    # La estructura exacta puede cambiar, así que es importante inspeccionar la página si el script falla.
    # Por el HTML que proporcionaste, parece que cada película está en un div con una clase específica,
    # que a su vez está dentro de elementos 'li' o directamente en un contenedor principal.
    # Vamos a buscar los contenedores de cada película.
    # El HTML proporcionado sugiere que cada película está en un <div class="sc-995e3276-1 jziSZL cli-parent li-compact">
    # Sin embargo, la lista principal suele estar en <ul> o <ol> o un div con un data-testid específico.
    # Busquemos la lista que parece ser la principal:
    movie_list_container = soup.find('ul', class_=['ipc-metadata-list', 'ipc-metadata-list--dividers-between', 'sc-71ed9118-0', 'grmHsY', 'compact-list-view', 'ipc-metadata-list--base'])
    
    if not movie_list_container:
        # Intento alternativo si la clase anterior no se encuentra (IMDb actualiza sus clases a veces)
        # El selector que has proporcionado es <div class="sc-995e3276-1 jziSZL cli-parent li-compact">
        # Estos elementos 'li' o 'div' suelen estar dentro de un contenedor mayor.
        # Si el anterior no funciona, intentamos encontrar todos los elementos que parecen ser una película individual.
        # Asumimos que la clase 'cli-parent' es un buen indicador.
        movie_elements = soup.select('li .cli-parent') # Buscamos 'cli-parent' dentro de un 'li'
        if not movie_elements: # Si no hay 'li .cli-parent', buscamos directamente 'cli-parent'
             movie_elements = soup.find_all('div', class_=re.compile(r'cli-parent'))


    else:
        movie_elements = movie_list_container.find_all('li', class_=re.compile(r'ipc-metadata-list-summary-item'))


    if not movie_elements:
        print("No se pudieron encontrar los elementos de las películas. La estructura de la página puede haber cambiado.")
        print("Por favor, revisa el HTML de la página https://www.imdb.com/es-es/chart/top/ y ajusta los selectores de BeautifulSoup.")
        # Imprimir una porción del HTML para ayudar a depurar si es necesario (opcional)
        # print(soup.prettify()[:2000])
        return None

    print(f"Se encontraron {len(movie_elements)} elementos de películas.")

    for movie_el in movie_elements:
        try:
            # --- Título y Posición ---
            # <h3 class="ipc-title__text">1. Cadena perpetua</h3>
            title_tag = movie_el.find('h3', class_='ipc-title__text')
            if title_tag:
                title_text = title_tag.get_text(strip=True)
                # Usar regex para separar posición y título
                match = re.match(r'(\d+)\.\s*(.+)', title_text)
                if match:
                    posicion = match.group(1)
                    titulo = match.group(2)
                else:
                    # Si no hay número, podría ser un formato diferente o no ser una película de la lista principal
                    titulo = title_text
                    posicion = "N/A" # O intentar obtenerlo de otra forma
            else:
                titulo = "N/A"
                posicion = "N/A"

            # --- Año, Duración ---
            # <div class="sc-4b408797-7 fUdAcX cli-title-metadata">
            #   <span class="sc-4b408797-8 iurwGb cli-title-metadata-item">1994</span>
            #   <span class="sc-4b408797-8 iurwGb cli-title-metadata-item">2h 22m</span>
            #   <span class="sc-4b408797-8 iurwGb cli-title-metadata-item">13</span>
            # </div>
            metadata_items = movie_el.find_all('span', class_='cli-title-metadata-item')
            if len(metadata_items) >= 2:
                ano = metadata_items[0].get_text(strip=True)
                duracion = metadata_items[1].get_text(strip=True)
                # El tercer elemento suele ser la clasificación por edad, que no pediste.
            else:
                ano = "N/A"
                duracion = "N/A"

            # --- Rating ---
            # <span aria-label="Calificación de IMDb: 9,3" class="ipc-rating-star ..."><span class="ipc-rating-star--rating">9,3</span>...</span>
            # O directamente buscar por la clase del span interno que contiene la calificación
            rating_tag = movie_el.find('span', class_='ipc-rating-star--rating')
            if rating_tag:
                rating = rating_tag.get_text(strip=True)
            else:
                # Intento alternativo con la estructura que diste originalmente
                rating_container = movie_el.find('div', class_='sc-bfa1b6a1-0') # o la clase más específica 'ezSnho'
                if rating_container:
                    rating_value_tag = rating_container.find('span', class_='ipc-rating-star--rating')
                    if rating_value_tag:
                         rating = rating_value_tag.get_text(strip=True)
                    else:
                        rating = "N/A"
                else:
                    rating = "N/A"
            
            # Validar que tenemos datos antes de añadir (evitar filas vacías si algo falla)
            if titulo != "N/A" and posicion != "N/A":
                titulos.append(titulo)
                posiciones.append(posicion)
                anos.append(ano)
                duraciones.append(duracion)
                ratings.append(rating)

        except Exception as e:
            print(f"Error procesando una película: {e}")
            # Puedes decidir si añadir N/A o saltar este elemento
            # titulos.append("Error")
            # ... y así para los demás campos

    if not titulos:
        print("No se pudo extraer ninguna información de las películas.")
        return None

    # Crear DataFrame
    df = pd.DataFrame({
        'Posición': posiciones,
        'Título': titulos,
        'Año': anos,
        'Duración': duraciones,
        'Rating': ratings
    })

    return df

if __name__ == '__main__':
    df_peliculas = scrape_imdb_top_250()
    if df_peliculas is not None:
        print("\n--- DataFrame de Películas ---")
        print(df_peliculas.head()) # Muestra las primeras 5 películas
        print(f"\nSe extrajeron {len(df_peliculas)} películas.")

        # Para ver todas las películas:
        # print(df_peliculas)

        # Para guardar en un CSV (opcional):
        # df_peliculas.to_csv('imdb_top_250.csv', index=False)
        # print("\nDataFrame guardado en imdb_top_250.csv")

Se encontraron 25 elementos de películas.

--- DataFrame de Películas ---
  Posición                 Título   Año Duración Rating
0        1        Cadena perpetua  1994   2h 22m    9,3
1        2             El padrino  1972   2h 55m    9,2
2        3    El caballero oscuro  2008   2h 32m    9,0
3        4    El padrino parte II  1974   3h 22m    9,0
4        5  12 hombres sin piedad  1957   1h 36m    9,0

Se extrajeron 25 películas.


: 